In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'C:\\Users\\Legion\\OneDrive\\Desktop\\Paris-Saclay\\Learning\\AI\\badminton-pose-coach'

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    raw_data_path: Path
    processed_data_path: Path
    params_fps: int
    params_conf: int
    params_keypoint_extraction_model: str

In [11]:
from badmintonPoseCoach.constants import *
from badmintonPoseCoach.utils.common import *

In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        params = self.params.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            raw_data_path = Path(config.raw_data_path),
            processed_data_path = Path(config.processed_data_path),
            params_fps = params.fps,
            params_conf = params.conf,
            params_keypoint_extraction_model = params.keypoint_extraction_model,
        )
        return data_ingestion_config

In [21]:
import os
from badmintonPoseCoach import logger
import os, json, pathlib
from typing import Iterable
from badmintonPoseCoach.utils.common import get_size, extract_keypoints_from_video

In [22]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def iter_videos(self, root: str) -> Iterable[str]:
        """Yield all video file paths under root recursively."""
        for dirpath, _, filenames in os.walk(root):
            for fn in filenames:
                if fn.lower().endswith("mp4"):
                    yield os.path.join(dirpath, fn)

    def mirror_and_save_json(self):
        """
        Walk through input_root/videos/<class>/*.mp4 and save JSON to
        output_root/<class>/<video_stem>.json, preserving the class folder.
        """
        input_root = os.path.abspath(self.config.raw_data_path)
        output_root = os.path.abspath(self.config.processed_data_path)

        for vp in self.iter_videos(input_root):
            # Compute relative path to preserve class folder
            rel = os.path.relpath(vp, input_root)              # e.g., "forehand/clip01.mp4"
            rel_no_ext = os.path.splitext(rel)[0]              # "forehand/clip01"
            out_json_path = os.path.join(output_root, rel_no_ext + ".json")

            # Ensure parent directory exists
            os.makedirs(os.path.dirname(out_json_path), exist_ok=True)

            # Skip if already extracted
            if os.path.exists(out_json_path):
                print(f"[SKIP] Exists: {out_json_path}")
                continue

            print(f"[EXTRACT] {vp} -> {out_json_path}")
            data = extract_keypoints_from_video(
                video_path=vp,
                model_or_path=self.config.params_keypoint_extraction_model,
                fps_sample=self.config.params_fps,
                conf=self.config.params_conf,
            )

            # Attach label from parent directory name (class folder)
            label = pathlib.Path(rel_no_ext).parts[0]  # first folder under input_root
            data["label"] = label
            data["video_relpath"] = rel.replace("\\", "/")

            with open(out_json_path, "w", encoding="utf-8") as f:
                json.dump(data, f)






In [24]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.mirror_and_save_json()
except Exception as e:
    raise e

[2025-09-17 22:04:29,843: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-17 22:04:29,845: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-17 22:04:29,845: INFO: common: created directory at: artifacts]
[2025-09-17 22:04:29,845: INFO: common: created directory at: artifacts/data_ingestion]
[SKIP] Exists: C:\Users\Legion\OneDrive\Desktop\Paris-Saclay\Learning\AI\badminton-pose-coach\artifacts\data_ingestion\00_Short Serve\2022-08-30_18-00-09_dataset_set1_009_001424_001452_A_00.json
[SKIP] Exists: C:\Users\Legion\OneDrive\Desktop\Paris-Saclay\Learning\AI\badminton-pose-coach\artifacts\data_ingestion\00_Short Serve\2022-08-30_18-00-09_dataset_set1_016_001916_001941_A_00.json
[SKIP] Exists: C:\Users\Legion\OneDrive\Desktop\Paris-Saclay\Learning\AI\badminton-pose-coach\artifacts\data_ingestion\00_Short Serve\2022-08-30_18-00-09_dataset_set1_023_002458_002481_A_00.json
[SKIP] Exists: C:\Users\Legion\OneDrive\Desktop\Paris-Saclay\Learning\AI\badmin

KeyboardInterrupt: 

In [9]:
obj = DataIngestion(None)
obj.mirror_and_save_json(
    input_root="data/VideoBadminton_Dataset/VideoBadminton_Dataset",
    output_root="artifacts/data_ingestion",
    model_or_path="yolo11n-pose.pt",
    fps_sample=15,
    conf=0.25,
)

[EXTRACT] C:\Users\Legion\OneDrive\Desktop\Paris-Saclay\Learning\AI\badminton-pose-coach\data\VideoBadminton_Dataset\VideoBadminton_Dataset\00_Short Serve\2022-08-30_18-00-09_dataset_set1_009_001424_001452_A_00.mp4 -> C:\Users\Legion\OneDrive\Desktop\Paris-Saclay\Learning\AI\badminton-pose-coach\artifacts\data_ingestion\00_Short Serve\2022-08-30_18-00-09_dataset_set1_009_001424_001452_A_00.json
[EXTRACT] C:\Users\Legion\OneDrive\Desktop\Paris-Saclay\Learning\AI\badminton-pose-coach\data\VideoBadminton_Dataset\VideoBadminton_Dataset\00_Short Serve\2022-08-30_18-00-09_dataset_set1_016_001916_001941_A_00.mp4 -> C:\Users\Legion\OneDrive\Desktop\Paris-Saclay\Learning\AI\badminton-pose-coach\artifacts\data_ingestion\00_Short Serve\2022-08-30_18-00-09_dataset_set1_016_001916_001941_A_00.json
[EXTRACT] C:\Users\Legion\OneDrive\Desktop\Paris-Saclay\Learning\AI\badminton-pose-coach\data\VideoBadminton_Dataset\VideoBadminton_Dataset\00_Short Serve\2022-08-30_18-00-09_dataset_set1_023_002458_00248

KeyboardInterrupt: 

In [1]:
from ultralytics import YOLO

In [8]:
model = YOLO("yolo11n-pose.pt")

In [5]:
print(model)

YOLO(
  (model): PoseModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats